In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import healpy as hp
from tqdm import tqdm, tqdm_notebook
import scipy as sp
from scipy.special import erf
from scipy import stats

import sys
from my_units import * 

from angular_fn import *
from template_fn import *
from cov_matrix_fn import *

import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable


HomeDir = './'
DataDir = '/Users/crimondino/Dropbox (PI)/LensVelocity2/data/pairs_ruwe_dist/'#HomeDir #+ 'code/'
### Set this to the directory where you store your data files (see below how to download them)
ListDir = HomeDir+'lists/hist_stats/'
FigDir = '../../../figures/'

In [ ]:
### Read in the data files for the background and foreground stars
df_back = pd.read_csv(DataDir+'bg_ruwe_dist.csv', 
                      usecols = ['source_id', 'ra', 'dec', 'pmra', 'pmdec','pmra_error', 'pmdec_error', 'pmra_pmdec_corr', 'pmra_eff_error', 'pmdec_eff_error', 'pmra_pmdec_eff_corr', 
                                 'parallax', 'parallax_error', 'pmra_sub', 'pmdec_sub', 'phot_g_mean_mag', 'ruwe'])
df_fore = pd.read_csv(DataDir+'fg_ruwe_dist.csv', 
                      usecols = ['source_id', 'ra', 'dec', 'pmra', 'pmdec','pmra_error', 'pmdec_error', 'parallax', 'parallax_error', 'phot_g_mean_mag', 'ruwe'])

                      
len(df_back), len(df_fore)

In [ ]:
### Read in the data files for the Bailer-Jones distances
back_dist = pd.read_csv(DataDir+'bg_distances.csv', usecols = ['0', '1', '2', '3'])
fore_dist = pd.read_csv(DataDir+'fg_distances.csv', usecols = ['0', '1', '2', '3'])
len(back_dist), len(fore_dist)

# Quality cuts

In [ ]:
max_ruwe = 1.4
nsigma = 2

In [ ]:
## cut out stars with ruwe > 1.4

df_fore = df_fore[df_back['ruwe'] < max_ruwe]
back_dist = back_dist[df_back['ruwe'] < max_ruwe]
fore_dist = fore_dist[df_back['ruwe'] < max_ruwe]

df_back = df_back[df_back['ruwe'] < max_ruwe]


df_back = df_back[df_fore['ruwe'] < max_ruwe]
back_dist = back_dist[df_fore['ruwe'] < max_ruwe]
fore_dist = fore_dist[df_fore['ruwe'] < max_ruwe]

df_fore = df_fore[df_fore['ruwe'] < max_ruwe]


print(len(df_fore))

In [ ]:
back_dist_med = back_dist['1'].to_numpy()
back_dist_lo = back_dist['2'].to_numpy()
back_dist_hi = back_dist['3'].to_numpy()

fore_dist_med = fore_dist['1'].to_numpy()
fore_dist_lo = fore_dist['2'].to_numpy()
fore_dist_hi = fore_dist['3'].to_numpy()

condition = fore_dist_med + nsigma * (fore_dist_hi - fore_dist_med) < back_dist_med - nsigma * (back_dist_med - back_dist_lo)

df_fore = df_fore[condition]
df_back = df_back[condition]

fore_dist = fore_dist[condition]
back_dist = back_dist[condition]

back_dist_med = back_dist['1'].to_numpy()
back_dist_lo = back_dist['2'].to_numpy()
back_dist_hi = back_dist['3'].to_numpy()

fore_dist_med = fore_dist['1'].to_numpy()
fore_dist_lo = fore_dist['2'].to_numpy()
fore_dist_hi = fore_dist['3'].to_numpy()
print(len(df_fore))

In [ ]:
df_back['dist'] = back_dist_med
# take average of posterior widths
df_back['dist_error'] = ((back_dist_med - back_dist_lo) + (back_dist_hi - back_dist_med)) / 2

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4),dpi=150)
ax.hist(fore_dist['1']/back_dist['1'], log = True,  density = True, bins = 100, histtype='step', color = 'darkslateblue')
ax.set_xlabel(r'Bailer-Jones distance [pc]')
fig.suptitle('Histograms of background distances')
plt.show()